<a href="https://colab.research.google.com/github/Bipolar-Ochka/AnimeBot/blob/master/test_stand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Импорты


In [1]:
import numpy as np





---



# 2. Вспомогательные функции

## 2.1. Текст в выбранной кодировке -> массив выбранного типа

По умолчанию кодировка - **windows-1251**

In [ ]:

def text2bin(text, encoding = 'windows-1251'):
    res = []
    for num in text.encode(encoding):
        res.extend(i for i in map(int, '{:08b}'.format(num)))
    
    return res

def gettextarray(text, encoding = 'windows-1251', type = 'bool'):
    return np.array(text2bin(text, encoding), dtype=type)